In [1]:
import glob
import sys
repo_dir = sys.path[0]

import xarray as xr
import numpy as np
import cmdline_provenance as cmdprov
import git
import zarr
import pandas as pd

# CAFE data

In [58]:
cafe_files = glob.glob('/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19??????/ZARR/atmos_isobaric_daily.zarr.zip')
cafe_files.sort()

In [59]:
cafe_files

['/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19841101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19851101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19861101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19871101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19881101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19891101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19901101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19911101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19921101/ZARR/atmos_isobaric_daily.zarr.zip',
 '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19931101/ZARR/at

## Open and clip a single file

In [60]:
ds = xr.open_zarr(cafe_files[0], consolidated=True, use_cftime=True)
ds

<xarray.Dataset>
Dimensions:         (ensemble: 96, lat: 90, latb: 91, level: 21, lon: 144, lonb: 145, nv: 2, time: 3652)
Coordinates: (12/14)
    area            (lat, lon) float32 dask.array<chunksize=(90, 144), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(3652,), meta=np.ndarray>
    average_T1      (time) object dask.array<chunksize=(3652,), meta=np.ndarray>
    average_T2      (time) object dask.array<chunksize=(3652,), meta=np.ndarray>
  * ensemble        (ensemble) int64 1 2 3 4 5 6 7 8 ... 89 90 91 92 93 94 95 96
  * lat             (lat) float64 -89.49 -87.98 -85.96 ... 85.96 87.98 89.49
    ...              ...
  * lon             (lon) float64 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
  * lonb            (lonb) float64 0.0 2.5 5.0 7.5 ... 352.5 355.0 357.5 360.0
  * nv              (nv) float64 1.0 2.0
  * time            (time) object 1984-11-01 12:00:00 ... 1994-10-31 12:00:00
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(3652, 2), meta=np.ndarray>
    zsurf           (lat, lon) float32 dask.array<chunksize=(90, 144), meta=np.ndarray>
Data variables: (12/38)
    DELP            (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    awp             (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    evap            (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    h500            (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    hght            (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    lwdn_sfc        (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    ...              ...
    u_ref           (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    ucomp           (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    v_ref           (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    vcomp           (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(28, 96, 1, 90, 144), meta=np.ndarray>
    vis_exopd_vl_c  (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
    wvp             (time, ensemble, lat, lon) float32 dask.array<chunksize=(28, 96, 90, 144), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    title:      AccessOcean-AM2

In [61]:
da = ds['precip']
for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'zsurf', 'area']:
    if drop_coord in da.coords:
        da = da.drop(drop_coord)

In [62]:
da

,Array,Chunk
Bytes,18.17 GB,139.35 MB
Shape,"(3652, 96, 90, 144)","(28, 96, 90, 144)"
Count,132 Tasks,131 Chunks
Type,float32,numpy.ndarray


In [63]:
da['time']

<xarray.DataArray 'time' (time: 3652)>
array([cftime.DatetimeJulian(1984, 11, 1, 12, 0, 0, 0),
       cftime.DatetimeJulian(1984, 11, 2, 12, 0, 0, 0),
       cftime.DatetimeJulian(1984, 11, 3, 12, 0, 0, 0), ...,
       cftime.DatetimeJulian(1994, 10, 29, 12, 0, 0, 0),
       cftime.DatetimeJulian(1994, 10, 30, 12, 0, 0, 0),
       cftime.DatetimeJulian(1994, 10, 31, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1984-11-01 12:00:00 ... 1994-10-31 12:00:00
Attributes:
    bounds:          time_bounds
    calendar_type:   JULIAN
    cartesian_axis:  T
    long_name:       time

Note: Each CAFE file has a decade of daily data. Some decades have two February 29ths (for a total of 3652 days) and other decades have three (for a total of 3653 days).

In [64]:
#da['time'] = da.indexes['time'].to_datetimeindex()

In [65]:
#da['time']

In [66]:
da['lat']

<xarray.DataArray 'lat' (lat: 90)>
array([-89.494382, -87.977528, -85.955056, -83.932584, -81.910112, -79.88764 ,
       -77.865169, -75.842697, -73.820225, -71.797753, -69.775281, -67.752809,
       -65.730337, -63.707865, -61.685393, -59.662921, -57.640449, -55.617978,
       -53.595506, -51.573034, -49.550562, -47.52809 , -45.505618, -43.483146,
       -41.460674, -39.438202, -37.41573 , -35.393258, -33.370787, -31.348315,
       -29.325843, -27.303371, -25.280899, -23.258427, -21.235955, -19.213483,
       -17.191011, -15.168539, -13.146067, -11.123596,  -9.101124,  -7.078652,
        -5.05618 ,  -3.033708,  -1.011236,   1.011236,   3.033708,   5.05618 ,
         7.078652,   9.101124,  11.123596,  13.146067,  15.168539,  17.191011,
        19.213483,  21.235955,  23.258427,  25.280899,  27.303371,  29.325843,
        31.348315,  33.370787,  35.393258,  37.41573 ,  39.438202,  41.460674,
        43.483146,  45.505618,  47.52809 ,  49.550562,  51.573034,  53.595506,
        55.617978,  57.640449,  59.662921,  61.685393,  63.707865,  65.730337,
        67.752809,  69.775281,  71.797753,  73.820225,  75.842697,  77.865169,
        79.88764 ,  81.910112,  83.932584,  85.955056,  87.977528,  89.494382])
Coordinates:
  * lat      (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
Attributes:
    cartesian_axis:  Y
    edges:           latb
    long_name:       latitude
    units:           degrees_N

In [67]:
da['lon']

<xarray.DataArray 'lon' (lon: 144)>
array([  1.25,   3.75,   6.25,   8.75,  11.25,  13.75,  16.25,  18.75,  21.25,
        23.75,  26.25,  28.75,  31.25,  33.75,  36.25,  38.75,  41.25,  43.75,
        46.25,  48.75,  51.25,  53.75,  56.25,  58.75,  61.25,  63.75,  66.25,
        68.75,  71.25,  73.75,  76.25,  78.75,  81.25,  83.75,  86.25,  88.75,
        91.25,  93.75,  96.25,  98.75, 101.25, 103.75, 106.25, 108.75, 111.25,
       113.75, 116.25, 118.75, 121.25, 123.75, 126.25, 128.75, 131.25, 133.75,
       136.25, 138.75, 141.25, 143.75, 146.25, 148.75, 151.25, 153.75, 156.25,
       158.75, 161.25, 163.75, 166.25, 168.75, 171.25, 173.75, 176.25, 178.75,
       181.25, 183.75, 186.25, 188.75, 191.25, 193.75, 196.25, 198.75, 201.25,
       203.75, 206.25, 208.75, 211.25, 213.75, 216.25, 218.75, 221.25, 223.75,
       226.25, 228.75, 231.25, 233.75, 236.25, 238.75, 241.25, 243.75, 246.25,
       248.75, 251.25, 253.75, 256.25, 258.75, 261.25, 263.75, 266.25, 268.75,
       271.25, 273.75, 276.25, 278.75, 281.25, 283.75, 286.25, 288.75, 291.25,
       293.75, 296.25, 298.75, 301.25, 303.75, 306.25, 308.75, 311.25, 313.75,
       316.25, 318.75, 321.25, 323.75, 326.25, 328.75, 331.25, 333.75, 336.25,
       338.75, 341.25, 343.75, 346.25, 348.75, 351.25, 353.75, 356.25, 358.75])
Coordinates:
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
Attributes:
    cartesian_axis:  X
    edges:           lonb
    long_name:       longitude
    units:           degrees_E

In [68]:
#AUS_BOX = [-46, -9, 111, 157]
AUS_BOX = [-44, -11, 113, 154]

def get_region(da, box):
    """Select grid points that fall within a lat/lon box.
    
    Args:
      da (xarray DataArray)
      box (array-like) : [south bound, north bound, east bound, west bound]
    
    """

    lat_south_bound, lat_north_bound, lon_east_bound, lon_west_bound = box
    assert -90 <= lat_south_bound <= 90, "Valid latitude range is [-90, 90]"
    assert -90 <= lat_north_bound <= 90, "Valid latitude range is [-90, 90]"
    assert lat_south_bound < lat_north_bound, "South bound greater than north bound"
    assert 0 <= lon_east_bound < 360, "Valid longitude range is [0, 360)"
    assert 0 <= lon_west_bound < 360, "Valid longitude range is [0, 360)"
    
    da = da.assign_coords({'lon': (da['lon'] + 360)  % 360})
        
    mask_lat = (da['lat'] > lat_south_bound) & (da['lat'] < lat_north_bound)
    if lon_east_bound < lon_west_bound:
        mask_lon = (da['lon'] > lon_east_bound) & (da['lon'] < lon_west_bound)
    else:
        mask_lon = (da['lon'] > lon_east_bound) | (da['lon'] < lon_west_bound)
    
    da = da.where(mask_lat & mask_lon, drop=True) 
        
    #if sort:
    #    da = da.sortby(lat_name).sortby(lon_name)
    #da.sel({'lat': slice(box[0], box[1]), 'lon': slice(box[2], box[3])})

    return da 

In [69]:
da = get_region(da, AUS_BOX)

/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


In [70]:
da

,Array,Chunk
Bytes,405.28 MB,3.11 MB
Shape,"(3652, 96, 17, 17)","(28, 96, 17, 17)"
Count,526 Tasks,131 Chunks
Type,float32,numpy.ndarray


In [71]:
da['lon']

<xarray.DataArray 'lon' (lon: 17)>
array([113.75, 116.25, 118.75, 121.25, 123.75, 126.25, 128.75, 131.25, 133.75,
       136.25, 138.75, 141.25, 143.75, 146.25, 148.75, 151.25, 153.75])
Coordinates:
  * lon      (lon) float64 113.8 116.2 118.8 121.2 ... 146.2 148.8 151.2 153.8

In [72]:
da['lat']

<xarray.DataArray 'lat' (lat: 17)>
array([-43.483146, -41.460674, -39.438202, -37.41573 , -35.393258, -33.370787,
       -31.348315, -29.325843, -27.303371, -25.280899, -23.258427, -21.235955,
       -19.213483, -17.191011, -15.168539, -13.146067, -11.123596])
Coordinates:
  * lat      (lat) float64 -43.48 -41.46 -39.44 -37.42 ... -15.17 -13.15 -11.12
Attributes:
    cartesian_axis:  Y
    edges:           latb
    long_name:       latitude
    units:           degrees_N

## Open and clip (and stack) a bunch of files

In [73]:
def open_and_clip(infile):
    """Open file and select region"""
    
    ds = xr.open_zarr(infile, consolidated=True, use_cftime=True)
    
    da = ds['precip']
    for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'zsurf', 'area']:
        if drop_coord in da.coords:
            da = da.drop(drop_coord)
    
    da = get_region(da, AUS_BOX)
    
    return da

In [74]:
datasets = []
for infile in cafe_files[0:4]:
    da = open_and_clip(infile)
    lead_time = range(len(da['time']))
    init_date = np.datetime64(da['time'].values[0].strftime('%Y-%m-%d'))            
    datasets.append(da.rename({'time': 'lead_time'}).assign_coords({'lead_time': lead_time, 'init_date': init_date}))
da_cafe = xr.concat(datasets, dim='init_date')

/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/g/data/e14/dbi599/miniconda3/envs/unseen/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     

In [75]:
da_cafe

,Array,Chunk
Bytes,1.62 GB,3.11 MB
Shape,"(4, 3653, 96, 17, 17)","(1, 28, 96, 17, 17)"
Count,4465 Tasks,524 Chunks
Type,float32,numpy.ndarray


In [76]:
time_values = [da_cafe.get_index('init_date').shift(int(lead), 'D') for lead in da_cafe['lead_time']]
target_times = xr.DataArray(time_values,
                            dims={'lead_time': da_cafe['lead_time'],
                                  'init_date': da_cafe['init_date']})
da_cafe['lead_time'].attrs['units'] = 'D'
da_cafe = da_cafe.assign_coords({'time': target_times})

In [77]:
da_cafe

,Array,Chunk
Bytes,1.62 GB,3.11 MB
Shape,"(4, 3653, 96, 17, 17)","(1, 28, 96, 17, 17)"
Count,4465 Tasks,524 Chunks
Type,float32,numpy.ndarray


In [78]:
da_cafe['lead_time']

<xarray.DataArray 'lead_time' (lead_time: 3653)>
array([   0,    1,    2, ..., 3650, 3651, 3652])
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3647 3648 3649 3650 3651 3652
Attributes:
    units:    D

In [79]:
cafe_outfile = '/g/data/xv83/dbi599/precip_c5-d60-pX-f6_19841101-19871101_atmos_isobaric_daily.zarr.zip'
ds_cafe = da_cafe.to_dataset()

In [80]:
print(repo_dir)

/home/599/dbi599/unseen


In [81]:
repo = git.Repo(repo_dir)
repo_url = repo.remotes[0].url.split('.git')[0]
new_log = cmdprov.new_log(code_url=repo_url)
ds_cafe.attrs['history'] = new_log
print(new_log)

Wed Apr 21 16:25:04 2021: /g/data/e14/dbi599/miniconda3/envs/unseen/bin/jupyter notebook /home/599/dbi599/unseen/preprocessing.ipynb (https://github.com/ClimResAus/unseen)


In [82]:
ds_cafe

,Array,Chunk
Bytes,1.62 GB,3.11 MB
Shape,"(4, 3653, 96, 17, 17)","(1, 28, 96, 17, 17)"
Count,4465 Tasks,524 Chunks
Type,float32,numpy.ndarray


In [83]:
#ds.to_zarr(cafe_outfile, mode='w')
with zarr.ZipStore(cafe_outfile, mode='w') as store:
    ds_cafe.to_zarr(store)

**TODO**: The dimensions need more metadata

# JRA-55

In [84]:
jra_file = '/g/data/xv83/ds0092/data/csiro-dcfp-jra55/surface_daily_cafe-grid.zarr/'

In [85]:
ds_jra = xr.open_zarr(jra_file, consolidated=True, use_cftime=True)

In [86]:
ds_jra

<xarray.Dataset>
Dimensions:              (initial_time0_hours: 22950, lat: 90, lon: 144, lv_ULN1: 3)
Coordinates:
  * initial_time0_hours  (initial_time0_hours) object 1958-01-01 09:00:00 ......
  * lat                  (lat) float64 -89.49 -87.98 -85.96 ... 87.98 89.49
  * lon                  (lon) float64 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
  * lv_ULN1              (lv_ULN1) float64 1.0 2.0 3.0
Data variables: (12/15)
    DEPR_GDS0_HTGL       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    POT_GDS0_SFC         (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    PRES_GDS0_SFC        (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    PRMSL_GDS0_MSL       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    RH_GDS0_HTGL         (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    SPFH_GDS0_HTGL       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    ...                   ...
    TPRAT_GDS0_SFC       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    UGRD_GDS0_HTGL       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    ULWRF_GDS0_NTAT      (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    ULWRF_GDS0_SFC       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    VGRD_GDS0_HTGL       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
    WSMX_GDS4_HTGL       (initial_time0_hours, lat, lon) float32 dask.array<chunksize=(731, 90, 144), meta=np.ndarray>
Attributes:
    filename:  surface_daily_cafe-grid.zarr

In [88]:
ds_jra = ds_jra.rename({'initial_time0_hours': 'time'})
da_jra = ds_jra['TPRAT_GDS0_SFC']
da_jra = da_jra.rename('precip')

In [89]:
da_jra

<xarray.DataArray 'precip' (time: 22950, lat: 90, lon: 144)>
dask.array<xarray-TPRAT_GDS0_SFC, shape=(22950, 90, 144), dtype=float32, chunksize=(731, 90, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1958-01-01 09:00:00 ... 2020-10-31 09:00:00
  * lat      (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
Attributes:
    cell_methods:             initial_time0_hours: mean
    center:                   Japanese Meteorological Agency - Tokyo (RSMC)
    gds_grid_type:            0
    level_indicator:          1
    long_name:                Total precipitation
    parameter_number:         61
    parameter_table_version:  200
    sub_center:               241
    units:                    mm/day

Note: JRA-55 has 29 Feb in leap years.

In [94]:
da_jra = get_region(da_jra, AUS_BOX)
da_jra

<xarray.DataArray 'precip' (time: 22950, lat: 17, lon: 17)>
dask.array<where, shape=(22950, 17, 17), dtype=float32, chunksize=(731, 17, 17), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1958-01-01 09:00:00 ... 2020-10-31 09:00:00
  * lat      (lat) float64 -43.48 -41.46 -39.44 -37.42 ... -15.17 -13.15 -11.12
  * lon      (lon) float64 113.8 116.2 118.8 121.2 ... 146.2 148.8 151.2 153.8
Attributes:
    cell_methods:             initial_time0_hours: mean
    center:                   Japanese Meteorological Agency - Tokyo (RSMC)
    gds_grid_type:            0
    level_indicator:          1
    long_name:                Total precipitation
    parameter_number:         61
    parameter_table_version:  200
    sub_center:               241
    units:                    mm/day

In [96]:
period = (np.datetime64('1990-01-01'), np.datetime64('2020-12-31'))

stack_dates = da_cafe.sel(init_date=slice((pd.to_datetime(period[0]) - pd.DateOffset(years=10)).strftime('%Y-%m-%d'),
                          pd.to_datetime(period[1]).strftime('%Y-%m-%d'))).init_date.values

In [97]:
stack_dates

array(['1984-11-01T00:00:00.000000000', '1985-11-01T00:00:00.000000000',
       '1986-11-01T00:00:00.000000000', '1987-11-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [98]:
def stack_by_init_date(da, init_dates, N_lead_steps, init_date_name='init_date', 
                       lead_time_name='lead_time', time_name='time',
                       tolerance='D'):
    """ Stacks provided timeseries array in an inital date / lead time format. Note this process \
        replicates data and can substantially increase memory usage. Lead time frequency will match \
       frequency of input data. Returns nans if requested times lie outside of the available range.
    """

    # Convert cftimes to nptimes
    if xr.core.common.contains_cftime_datetimes(da[time_name]):
        times_np = xr.coding.times.cftime_to_nptime(da[time_name])
    else:
        times_np = da[time_name]
    
    #if xr.core.common.contains_cftime_datetimes(init_dates):
    #    init_dates_np = xr.coding.times.cftime_to_nptime(init_dates)
    #else:
    init_dates_np = init_dates
        
    freq = pd.infer_freq(times_np)
    if freq is None:
        raise ValueError('Cannot determine frequency of input timeseries')
    
    init_list = []
    for i in range(len(init_dates)):
        start_index = np.where(times_np == init_dates_np[i])[0]

        if start_index.size == 0:
            end_time = np.datetime64(pd.DatetimeIndex([init_dates_np[i]]).shift(N_lead_steps, freq=freq)[0])
            end_index = np.where(times_np.astype(f'datetime64[{tolerance}]') == 
                                 end_time.astype(f'datetime64[{tolerance}]'))[0]
            if end_index.size == 0:
                # Try step back from the end time - this can help with strange frequencies eg Q-DEC
                start_time = np.datetime64(pd.DatetimeIndex([end_time]).shift(-N_lead_steps, freq=freq)[0])
                start_index = np.where(times_np.astype(f'datetime64[{tolerance}]') == 
                                       start_time.astype(f'datetime64[{tolerance}]'))[0]
                
                if start_index.size == 0:
                    da_slice = da.isel({time_name:range(0, N_lead_steps)}).where(False, np.nan)
                    time_slice = da_slice[time_name].where(False, np.nan) \
                                                    .expand_dims({init_date_name: [init_dates[i].item()]})
                    init_list.append(da_slice.expand_dims({init_date_name: [init_dates[i].item()]}) \
                                             .assign_coords({f'{time_name}_new': time_slice}) \
                                             .assign_coords({time_name: np.arange(0, N_lead_steps)}) \
                                             .rename({time_name: lead_time_name}) \
                                             .rename({f'{time_name}_new': time_name}))
                else:
                    start_index = start_index.item()
                    da_slice = da.isel({time_name:slice(start_index, None)})
                    time_slice = da_slice[time_name].expand_dims({init_date_name: [init_dates[i].item()]})
                    init_list.append(da_slice.expand_dims({init_date_name: [init_dates[i].item()]}) \
                                             .assign_coords({f'{time_name}_new': time_slice}) \
                                             .assign_coords({time_name: np.arange(0, len(time_slice[time_name]))}) \
                                             .rename({time_name: lead_time_name}) \
                                             .rename({f'{time_name}_new': time_name}))
            else:
                end_index = end_index.item()
                start_index = end_index - N_lead_steps
                if start_index < 0:
                    hang = -start_index
                    start_index = 0
                else: hang = 0
                    
                da_slice = da.isel({time_name:range(start_index, end_index)})
                time_slice = da_slice[time_name].expand_dims({init_date_name: [init_dates[i].item()]})
                init_list.append(da_slice.expand_dims({init_date_name: [init_dates[i].item()]}) \
                                         .assign_coords({f'{time_name}_new': time_slice}) \
                                         .assign_coords({time_name: np.arange(0, end_index-start_index)+hang}) \
                                         .rename({time_name: lead_time_name}) \
                                         .rename({f'{time_name}_new': time_name}))
        else:
            start_index = start_index.item()
            end_index = min([start_index + N_lead_steps, len(times_np)])
            
            da_slice = da.isel({time_name:range(start_index, end_index)})
            time_slice = da_slice[time_name].expand_dims({init_date_name: [init_dates[i].item()]})
            init_list.append(da_slice.expand_dims({init_date_name: [init_dates[i].item()]}) \
                                     .assign_coords({f'{time_name}_new': time_slice}) \
                                     .assign_coords({time_name: np.arange(0, end_index-start_index)}) \
                                     .rename({time_name: lead_time_name}) \
                                     .rename({f'{time_name}_new': time_name}))
            
    stacked = xr.concat(init_list, dim=init_date_name)
    stacked[lead_time_name].attrs['units'] = freq
    
    return stacked

In [103]:
N_lead_steps = max(da_cafe.lead_time.values) + 1 
N_lead_steps

3653

In [105]:
da_jra = stack_by_init_date(da_jra, stack_dates, N_lead_steps)

In [106]:
da_jra

,Array,Chunk
Bytes,16.89 MB,421.94 kB
Shape,"(4, 3653, 17, 17)","(1, 365, 17, 17)"
Count,428 Tasks,84 Chunks
Type,float32,numpy.ndarray


In [136]:
jra_outfile = '/g/data/xv83/dbi599/precip_jra55_19841101-19871101_surface_daily_cafe-grid.zarr.zip'
ds_jra = da_jra.to_dataset()

In [137]:
ds_jra.attrs['history'] = new_log

In [138]:
ds_jra = ds_jra.chunk({'init_date': -1, 'lead_time': -1, 'lat': -1, 'lon': -1})

In [139]:
ds_jra.to_zarr(jra_outfile, mode='w')
#with zarr.ZipStore(jra_outfile, mode='w') as store:
#    ds_jra.to_zarr(store)

# AWAP

In [111]:
awap_file = '/g/data/xv83/ds0092/data/csiro-dcfp-csiro-awap/rain_day_19000101-20201202_cafe-grid.zarr/'

In [112]:
ds_awap = xr.open_zarr(awap_file, consolidated=True, use_cftime=True)

In [113]:
ds_awap

<xarray.Dataset>
Dimensions:   (lat: 17, lon: 17, nv: 2, time: 44166)
Coordinates:
  * lat       (lat) float64 -43.48 -41.46 -39.44 -37.42 ... -15.17 -13.15 -11.12
  * lon       (lon) float64 113.8 116.2 118.8 121.2 ... 146.2 148.8 151.2 153.8
  * time      (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Dimensions without coordinates: nv
Data variables:
    crs       (time, lat, lon) float64 dask.array<chunksize=(44166, 17, 17), meta=np.ndarray>
    lat_bnds  (time, lat, lon, nv) float64 dask.array<chunksize=(44166, 17, 17, 2), meta=np.ndarray>
    lon_bnds  (time, lat, lon, nv) float64 dask.array<chunksize=(44166, 17, 17, 2), meta=np.ndarray>
    precip    (lat, lon, time) float64 dask.array<chunksize=(17, 17, 44166), meta=np.ndarray>

In [114]:
da_awap = ds_awap['precip']

In [115]:
da_awap

<xarray.DataArray 'precip' (lat: 17, lon: 17, time: 44166)>
dask.array<xarray-precip, shape=(17, 17, 44166), dtype=float64, chunksize=(17, 17, 44166), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -43.48 -41.46 -39.44 -37.42 ... -15.17 -13.15 -11.12
  * lon      (lon) float64 113.8 116.2 118.8 121.2 ... 146.2 148.8 151.2 153.8
  * time     (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm
    valid_range:                   [-1.0, 100000.0]

In [128]:
da_awap = stack_by_init_date(da_awap, stack_dates, N_lead_steps)
da_awap

<xarray.DataArray 'precip' (init_date: 4, lat: 17, lon: 17, lead_time: 3653)>
dask.array<concatenate, shape=(4, 17, 17, 3653), dtype=float64, chunksize=(1, 17, 17, 3653), chunktype=numpy.ndarray>
Coordinates:
  * init_date  (init_date) int64 468115200000000000 ... 562723200000000000
  * lat        (lat) float64 -43.48 -41.46 -39.44 ... -15.17 -13.15 -11.12
  * lon        (lon) float64 113.8 116.2 118.8 121.2 ... 146.2 148.8 151.2 153.8
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3647 3648 3649 3650 3651 3652
    time       (init_date, lead_time) object 1984-11-01 09:00:00 ... 1997-10-...
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm
    valid_range:                   [-1.0, 100000.0]

In [129]:
awap_outfile = '/g/data/xv83/dbi599/precip_awap_19841101-19871101_daily_cafe-grid.zarr.zip'
ds_awap = da_awap.to_dataset()

In [130]:
ds_awap.attrs['history'] = new_log

In [131]:
#ds.to_zarr(awap_outfile, mode='w')
with zarr.ZipStore(awap_outfile, mode='w') as store:
    ds_awap.to_zarr(store)